In [2]:
import os
import glob
import numpy as np
import cv2
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
from keras import backend as K

Using TensorFlow backend.


In [3]:
K.set_image_data_format('channels_first')

In [4]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
def triplet_loss(y_true, y_pred, alpha = 0.3):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,
               positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, 
               negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
   
    return loss

In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [9]:
# for file in glob.glob("images/*"):
#     print(os.path.splitext(os.path.basename(file))[0])
# #         identity = os.path.splitext(os.path.basename(file))[0]

### We are only going to use one image of each individual in our implementation. The reason is that the FaceNet network is powerful enough to only need one image of an individual to recognise them!

In [7]:
path = "Dataset"

In [8]:
def prepare_database():
    path = "Dataset"
    database = {}
    for root, sub,filename in os.walk(path):
        for file in filename:
            cur_path = os.path.join(root,file)     

            identity = cur_path.split("\\")[1]
            database[identity] = img_path_to_encoding(cur_path, FRmodel)
    return database

In [9]:
def prepare_database():
    database = {}
    for file in glob.glob("images/*"):
        identity = os.path.splitext(os.path.basename(file))[0]
        database[identity] = img_path_to_encoding(file, FRmodel)
    return database

In [10]:
def webcam_face_recognizer(database):
   
    global ready_to_detect_identity

    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(0)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    while vc.isOpened():
        ret, frame = vc.read()
        if ret:
            img = frame

            # We do not want to detect a new identity while the program is in the process of identifying another person
            if ready_to_detect_identity:
                img = process_frame(img, frame, face_cascade)   

            key = cv2.waitKey(100)
            cv2.imshow("preview", img)

            if key == 27: # exit on ESC
                break
    cv2.destroyWindow("preview")


In [11]:
from keras import backend as K
import time
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import win32com.client as wincl

In [12]:
PADDING = 25
ready_to_detect_identity = True
windows10_voice_interface = wincl.Dispatch("SAPI.SpVoice")

In [13]:
def process_frame(img, frame, face_cascade):
    
    global ready_to_detect_identity
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop through all the faces detected and determine whether or not they are in the database
    identities = []
    for (x, y, w, h) in faces:
        x1 = x-PADDING
        y1 = y-PADDING
        x2 = x+w+PADDING
        y2 = y+h+PADDING

        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),2)

        identity = find_identity(frame, x1, y1, x2, y2)

        if identity is not None:
            identities.append(identity)

    if identities != []:
        cv2.imwrite('example.png',img)

        ready_to_detect_identity = False
        pool = Pool(processes=1) 
        # We run this as a separate process so that the camera feedback does not freeze
        pool.apply_async(welcome_users, [identities])
    return img

def find_identity(frame, x1, y1, x2, y2):
 
    height, width, channels = frame.shape
    # The padding is necessary since the OpenCV face detector creates the bounding box around the face and not the head
    part_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
#     part_image = frame[(y1):(height), ( x1):(width)]
    
#     print( who_is_it(part_image, database, FRmodel))
    return who_is_it(part_image, database, FRmodel)

In [23]:
# face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# testpath = "C:/Users/spari/Projects/SDKART/ACE/obama.jpg"
# img = cv2.imread(testpath)
# img = cv2.resize(img, (96, 96)) 
# # img = image[...,::-1]


# img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
# # cv2.imwrite('iee.png',img)
# # x_train = np.array([img])
# # embedding = model.predict_on_batch(x_train)
# # process_frame(img,img,face_cascade)
# img.shape

(3, 96, 96)

In [88]:
def who_is_it(image, database, model):
    encoding = img_to_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(db_enc - encoding)
        print('distance for %s is %s' %(name, dist))
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.70:
        return "No one"
    else:
        print("IDD", identity)
        cv2.putText(image,identity, (80,80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        return identity

In [34]:
def welcome_users(identities):
    """ Outputs a welcome audio message to the users """
    global ready_to_detect_identity
    welcome_message = 'Welcome '

    if len(identities) == 1:
        welcome_message += '%s, have a nice day.' % identities[0]
    else:
        for identity_id in range(len(identities)-1):
            welcome_message += '%s, ' % identities[identity_id]
        welcome_message += 'and %s, ' % identities[-1]
        welcome_message += 'have a nice day!'

    windows10_voice_interface.Speak(welcome_message)

    # Allow the program to start detecting identities again
    ready_to_detect_identity = True

# if __name__ == "__main__":
#     database = prepare_database()
#     webcam_face_recognizer(database)

In [145]:
database = prepare_database()
# webcam_face_recognizer(database)

In [138]:
webcam_face_recognizer(database)

distance for Pakhi is 0.848562
distance for Prateek is 0.873412
distance for Rishabh is 0.835415
distance for test is 0.818318
distance for Pakhi is 0.896077
distance for Prateek is 0.926984
distance for Rishabh is 0.786255
distance for test is 0.902738
distance for Pakhi is 0.969004
distance for Prateek is 0.914785
distance for Rishabh is 0.875001
distance for test is 0.961715
distance for Pakhi is 0.810441
distance for Prateek is 0.815598
distance for Rishabh is 0.774394
distance for test is 0.976657
distance for Pakhi is 0.83163
distance for Prateek is 0.860425
distance for Rishabh is 0.716925
distance for test is 0.935602
distance for Pakhi is 0.879699
distance for Prateek is 0.956492
distance for Rishabh is 0.851071
distance for test is 0.891471
distance for Pakhi is 0.864096
distance for Prateek is 0.872378
distance for Rishabh is 0.751984
distance for test is 0.927888
distance for Pakhi is 0.750772
distance for Prateek is 0.910714
distance for Rishabh is 0.758276
distance for te